# Summary of stuff

## Model Data Import

In [51]:
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

import os
import numpy as np
import pickle
import pandas as pd

In [2]:
model_df = pd.read_csv(r"..\data\training_data_eda.csv")

In [3]:
model_df = model_df.drop(columns="Unnamed: 0")

In [4]:
model_df.head()

,max_credit,gender,education,marital_status,age,pay_status_sep,pay_status_aug,pay_status_jul,pay_status_jun,pay_status_may,...,average_bill_change,full_pay_sep,full_pay_aug,full_pay_jul,full_pay_jun,full_pay__may,full_pay_apr,pay_status_sum,habit_delay,default
0,220000,1,1,2,36,0,0,0,0,0,...,7598.6,0,0,0,0,0,0,0,0,1
1,200000,1,3,2,29,-1,-1,-1,-1,-1,...,0.0,0,0,0,0,0,0,-6,0,0
2,180000,1,1,2,27,-2,-2,-2,-2,-2,...,0.0,1,1,1,1,1,1,-12,0,0
3,80000,0,2,2,32,0,0,0,0,0,...,1769.0,0,0,0,0,0,0,0,0,0
4,10000,0,2,2,27,0,0,0,0,0,...,1112.0,0,0,0,0,0,0,0,0,1


## Upsample Minority Result

In [5]:
model_df.groupby("default").age.count()

default
0    17471
1     5028
Name: age, dtype: int64

In [6]:
defaulted = model_df[model_df.default == 1]
undefaulted = model_df[model_df.default == 0]

In [7]:
defaulted_upsampled = resample(defaulted,
                          replace=True, # sample with replacement
                          n_samples=undefaulted.shape[0], # match number in majority class
                          random_state=42) # reproducible result

In [8]:
resampled_df  = pd.concat([undefaulted, defaulted_upsampled])
resampled_df.default.value_counts()

1    17471
0    17471
Name: default, dtype: int64

In [9]:
X = resampled_df.drop(columns="default")
y = resampled_df.default

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=40)

## Decision Tree GridSearchCV
This is gonna be a long one

In [54]:
tree = DecisionTreeClassifier()

In [44]:
# run history
# DecisionTreeClassifier(criterion='entropy', max_depth=22, max_features=39, min_samples_leaf=30)
# DecisionTreeClassifier(criterion='entropy', max_depth=18, max_features=47, min_samples_leaf=20)
tree_param_dict = {"max_depth":range(15,25,1),
              "criterion":["entropy"], 
              "min_samples_leaf":range(50,80,5),
              "splitter":["best"],
              "max_features":range(30, len(model_df.columns))} 

In [31]:
grid_tree = GridSearchCV(tree, tree_param_dict, cv=10, scoring="f1", n_jobs=-1, verbose=1)

In [45]:
grid_tree.fit(X_train, y_train)

Fitting 10 folds for each of 396 candidates, totalling 3960 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    8.9s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   36.4s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed:  6.0min
[Parallel(n_jobs=-1)]: Done 2442 tasks      | elapsed:  8.4min
[Parallel(n_jobs=-1)]: Done 3192 tasks      | elapsed: 11.2min
[Parallel(n_jobs=-1)]: Done 3960 out of 3960 | elapsed: 14.3min finished


In [46]:
# gridsearch decision tree parameter result
print(grid_tree.best_params_)
print(grid_tree.best_estimator_)
# make predictions for train and test
grid_tree_pred_train = grid_tree.best_estimator_.predict(X_train)
grid_tree_pred_test = grid_tree.best_estimator_.predict(X_test)

#check for significant difference in F1 between train and test for overfitting
print("Overfitting Check")
print("Train GridSearch DecisionTree F1 score: ", f1_score(y_train, grid_tree_pred_train))
print("Test GridSearch DecisionTree F1 score: ", f1_score(y_test, grid_tree_pred_test))

{'criterion': 'entropy', 'max_depth': 20, 'max_features': 51, 'min_samples_leaf': 10, 'splitter': 'best'}
DecisionTreeClassifier(criterion='entropy', max_depth=20, max_features=51,
                       min_samples_leaf=10)
Overfitting Check
Train GridSearch DecisionTree F1 score:  0.8844993968636912
Test GridSearch DecisionTree F1 score:  0.7994763255509492


In [61]:
avg_train_f1 = 0
avg_test_f1 = 0
for i in range(0, 100):
    # Get training set
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    
    grid_tree_pred_train = grid_tree.best_estimator_.predict(X_train)
    v = grid_tree.best_estimator_.predict(X_test)
    
    avg_train_f1 += f1_score(y_train, grid_tree_pred_train)
    avg_test_f1 += f1_score(y_test, grid_tree_pred_test)
    
#check for significant difference in F1 between train and test for overfitting
print("Overfitting Check")
print("Train GridSearch DecisionTree F1 score: ", avg_train_f1/100)
print("Test GridSearch DecisionTree F1 score: ", avg_test_f1/100)

Overfitting Check
Train GridSearch DecisionTree F1 score:  0.8624406496375339
Test GridSearch DecisionTree F1 score:  0.522740901108458


### Saving Models Through Pickle

In [60]:
# GridSearchCV Decision Tree Pickel
with open(r"..\model\grid_tree.pickle", "wb") as grid_tree_pickle:
    pickle.dump(grid_tree, grid_tree_pickle)

## Random Forrest Model

In [65]:
rfc = RandomForestClassifier(verbose=1)

In [66]:
rf_param_dict = {"criterion":["gini", "entropy"],
                 "max_depth":range(15,26,5), 
                 "min_samples_leaf":range(50, 81,10),
                 "max_features":range(30, len(model_df.columns), 2),
                 "n_estimators":range(100,251,50)
                 }

In [67]:
grid_rfc = GridSearchCV(rfc, rf_param_dict, cv=5, scoring="f1", n_jobs=-1, verbose=1)

In [ ]:
grid_rfc.fit(X_train, y_train)

Fitting 5 folds for each of 1056 candidates, totalling 5280 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  5.7min


In [ ]:
# gridsearch decision tree parameter result
print(grid_rfc.best_params_)
print(grid_rfc.best_estimator_)
# make predictions for train and test
grid_tree_pred_train = grid_rfc.best_estimator_.predict(X_train)
grid_tree_pred_test = grid_rfc.best_estimator_.predict(X_test)

#check for significant difference in F1 between train and test for overfitting
print("Overfitting Check")
print("Train GridSearch DecisionTree F1 score: ", f1_score(y_train, grid_tree_pred_train))
print("Test GridSearch DecisionTree F1 score: ", f1_score(y_test, grid_tree_pred_test))